In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns;
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
#accuracy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#pca
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
#clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.metrics import mean_absolute_error
#models
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
import os
print(os.listdir("../input/walmarts"))

['train.csv', 'sampleSubmission.csv', 'features.csv', 'stores.csv', 'test.csv']


In [ ]:
features=pd.read_csv("../input/walmarts/features.csv",sep=',', header=0,
                       names=['Store','Date','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4',
                              'MarkDown5','CPI','Unemployment','IsHoliday']).drop(columns=['IsHoliday'])
data=pd.read_csv("../input/walmarts/train.csv", names=['Store','Dept','Date','weeklySales','isHoliday'],sep=',', header=0)
stores=pd.read_csv("../input/walmarts/stores.csv", names=['Store','Type','Size'],sep=',', header=0)



In [ ]:
data=data.merge(stores, how='left').merge(features, how='left')


In [ ]:
data= pd.get_dummies(data, columns=["Type"])

In [ ]:
data[['MarkDown1','MarkDown2','MarkDown3','MarkDown4', 'MarkDown5']] = data[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].fillna(0)

In [ ]:
data['Month'] = pd.to_datetime(data['Date']).dt.month
data = data.drop(columns=["Date", "CPI", "Fuel_Price", 'Unemployment', 'MarkDown3'])

In [ ]:
data.head()

,Store,Dept,weeklySales,isHoliday,Size,Temperature,MarkDown1,MarkDown2,MarkDown4,MarkDown5,Type_A,Type_B,Type_C,Month
0,1,1,24924.50,False,151315,42.31,0.0,0.0,0.0,0.0,1,0,0,2
1,1,1,46039.49,True,151315,38.51,0.0,0.0,0.0,0.0,1,0,0,2
2,1,1,41595.55,False,151315,39.93,0.0,0.0,0.0,0.0,1,0,0,2
3,1,1,19403.54,False,151315,46.63,0.0,0.0,0.0,0.0,1,0,0,2
4,1,1,21827.90,False,151315,46.50,0.0,0.0,0.0,0.0,1,0,0,3


In [ ]:
def knn():
    knn = KNeighborsRegressor(n_neighbors=10)
    return knn
def extraTreesRegressor():
    clf = ExtraTreesRegressor(n_estimators=100,max_features='auto', verbose=1, n_jobs=1)
    return clf
def randomForestRegressor():
    clf = RandomForestRegressor(n_estimators=100,max_features='log2', verbose=1)
    return clf
def XGB():
    clf = XGBRegressor(verbosity=0)
    return clf
def svm():
    clf = SVR(kernel='rbf', gamma='auto')
    return clf
def model_():
    return randomForestRegressor()

def predict_(m, test_x):
    return pd.Series(m.predict(test_x))


def train_(train_x, train_y):
    m = model_()
    m.fit(train_x, train_y)
    return m

def train_and_predict(train_x, train_y, test_x):
    m = train_(train_x, train_y)
    return predict_(m, test_x), m
#
def calculate_error(test_y, predicted, weights):
    return mean_absolute_error(test_y, predicted, sample_weight=weights)

In [ ]:
kf = KFold(n_splits=5)
splited = []
# dataset2 = dataset.copy()
for name, group in data.groupby(["Store", "Dept"]):
    group = group.reset_index(drop=True)
    trains_x = []
    trains_y = []
    tests_x = []
    tests_y = []
    if group.shape[0] <= 5:
        f = np.array(range(5))
        np.random.shuffle(f)
        group['fold'] = f[:group.shape[0]]
        continue
    fold = 0
    for train_index, test_index in kf.split(group):
        group.loc[test_index, 'fold'] = fold
        fold += 1
    splited.append(group)

splited = pd.concat(splited).reset_index(drop=True)

In [ ]:
splited

,Store,Dept,weeklySales,isHoliday,Size,Temperature,MarkDown1,MarkDown2,MarkDown4,MarkDown5,Type_A,Type_B,Type_C,Month,fold
0,1,1,24924.50,False,151315,42.31,0.00,0.00,0.00,0.00,1,0,0,2,0.0
1,1,1,46039.49,True,151315,38.51,0.00,0.00,0.00,0.00,1,0,0,2,0.0
2,1,1,41595.55,False,151315,39.93,0.00,0.00,0.00,0.00,1,0,0,2,0.0
3,1,1,19403.54,False,151315,46.63,0.00,0.00,0.00,0.00,1,0,0,2,0.0
4,1,1,21827.90,False,151315,46.50,0.00,0.00,0.00,0.00,1,0,0,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421269,45,98,508.37,False,118221,64.88,4556.61,20.64,1601.01,3288.25,0,1,0,9,4.0
421270,45,98,628.10,False,118221,64.89,5046.74,0.00,2253.43,2340.01,0,1,0,10,4.0
421271,45,98,1061.02,False,118221,54.47,1956.28,0.00,599.32,3990.54,0,1,0,10,4.0
421272,45,98,760.01,False,118221,56.47,2004.02,0.00,437.73,1537.49,0,1,0,10,4.0


In [ ]:
best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    data = splited.loc[splited['fold'] != fold]
    test = splited.loc[splited['fold'] == fold]
    train_y = data['weeklySales']
    train_x = data.drop(columns=['weeklySales', 'fold'])
    test_y = test['weeklySales']
    test_x = test.drop(columns=['weeklySales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    predicted, model = train_and_predict(train_x, train_y, test_x)
    weights = test_x['isHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

(335722, 15) (85552, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   45.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished


0 3715.352616969697
Find best model
(335722, 15) (85552, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   44.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished


1 6222.382799011188
(335722, 15) (85552, 15)


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   41.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished


3 10438.723978309172
(335722, 15) (85552, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   41.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4 6021.518077487028


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished


In [ ]:
error_cv

6052.759301083852

In [ ]:
best_error

3715.352616969697

In [ ]:
test = pd.read_csv("../input/walmarts/test.csv", names=['Store','Dept','Date','isHoliday'],sep=',', header=0)
features=pd.read_csv("../input/walmarts/features.csv",sep=',', header=0,
                       names=['Store','Date','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4',
                              'MarkDown5','CPI','Unemployment','IsHoliday']).drop(columns=['IsHoliday'])

stores=pd.read_csv("../input/walmarts/stores.csv", names=['Store','Type','Size'],sep=',', header=0)
test = test.merge(stores, how='left').merge(features, how='left')

In [ ]:
test = pd.get_dummies(test, columns=["Type"])
test[['MarkDown1','MarkDown2','MarkDown3','MarkDown4', 'MarkDown5']] = test[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].fillna(0)
test = test.fillna(0)
column_date = test['Date']
test['Month'] = pd.to_datetime(test['Date']).dt.month
test = test.drop(columns=["Date", "CPI", "Fuel_Price", 'Unemployment', 'MarkDown3'])


In [ ]:
predicted_test = best_model.predict(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished


In [ ]:
test['weeklySales'] = predicted_test
test['Date'] = column_date
test['id'] = test['Store'].astype(str) + '_' +  test['Dept'].astype(str) + '_' +  test['Date'].astype(str)
test = test[['id', 'weeklySales']]
test = test.rename(columns={'id': 'Id', 'weeklySales': 'Weekly_Sales'})

In [ ]:
test.to_csv('output.csv', index=False)

In [ ]:
import pickle

In [ ]:
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(best_model, file)

In [ ]:
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

RandomForestRegressor(max_features='log2', verbose=1)

In [ ]:
score = Pickled_LR_Model.score(test_x, test_y)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score)) 

NameError: name 'Xtest' is not defined